# Identification of Plant Leaf diseases using Transfer Learning Method - Inception V3

**Importing necessary python libraries and Modules**

In [ ]:
#!pip install plotly
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.layers  import Concatenate
from tensorflow.keras.layers import AveragePooling2D, Dropout, Input, BatchNormalization
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Input, Add, Dense,GlobalAvgPool2D, Concatenate, AvgPool2D, Dropout, ReLU, Activation, MaxPool2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
import matplotlib.pyplot as plt # for ploting graph
import os

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
#For ROC
from sklearn.metrics import roc_curve 
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from time import perf_counter 

In [ ]:
import plotly.graph_objects as go_obj
from plotly.subplots import make_subplots
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

**Setting the path where train,valid and test directories are located**

In [ ]:
dataset_root = "/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"
train_path = os.path.join(dataset_root, "train")
valid_path = os.path.join(dataset_root, "valid")
test_path  = os.path.join("/kaggle/input/new-plant-diseases-dataset/test", "test")
diseases   = os.listdir(train_path) # name of the disease here

In [ ]:
batch_size=50 # batch size 

In [ ]:
#Setting the Traning dataset folder
training_set = ImageDataGenerator( rescale = 1/255.5)
training_data = training_set.flow_from_directory(
    train_path,
    target_size = (224,224),
    class_mode = "categorical",
    batch_size = batch_size
  )

In [ ]:
#Setting the validation dataset folder 
validation_set = ImageDataGenerator( rescale = 1/255.5)
validation_data = validation_set.flow_from_directory(
    valid_path,
    target_size = (224,224),
    class_mode = "categorical",
    shuffle = False,  # For corretly printing the ConfusionMartix and Roc curvs
    batch_size = batch_size
)

**Data exploration**

In [ ]:
#Finding the categories
categories = training_data.class_indices.keys()
print(categories)

In [ ]:
#getting the class name and displaying 16 images and its class name
classes=list(training_data.class_indices.keys())
plt.figure(figsize=(18,18))
for X_batch, y_batch in training_data:
    
    for i in range(0,16):
        plt.subplot(4,4,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    # show the plot
    plt.show()
   # plt.savefig('rnd.jpg', bbox_inches = 'tight')
    break

In [ ]:
#Finding Number of Uniqe plantes in the dataset
plants = []
NumberOfDiseases = 0
for plant in diseases:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        NumberOfDiseases += 1
print(f"Unique Plants are: \n{plants}\n")
# displayig number of unique diseases
print("Number of diseases are: {}".format(NumberOfDiseases))

In [ ]:
# Number of images for each disease
nums_train = {}
nums_val = {}
for disease in diseases:
    nums_train[disease] = len(os.listdir(train_path + '/' + disease))
    nums_val[disease] = len(os.listdir(valid_path + '/' + disease))
img_per_class_train = pd.DataFrame(nums_train.values(), index=nums_train.keys(), columns=["no. of images"])
print('Train datast distribution :')
img_per_class_train


In [ ]:
# section for visualising each class in the training raw dataset
train_diratory=train_path
nums = {}
for disease in diseases:
    nums[disease] = len(os.listdir(train_path + '/' + disease))

# plotting number of images available for each image classes
index = [n for n in range(38)]
plt.figure(figsize=(15, 5))
plt.bar(index, [n for n in nums.values()], width=0.4)
plt.xlabel('Plants/Diseases', fontsize=10)
plt.ylabel('No of images available', fontsize=10)
plt.xticks(index, diseases, fontsize=11, rotation=90)
plt.title('Images per each class of plant disease')


In [ ]:
def Display_Model_Loss_Accuracy(history):
  # funtion for ploting model accuracy and loss graph
  #=============================================================================
  # Create figure with secondary y-axis
  fig = make_subplots(specs=[[{"secondary_y": True}]])

  # Adding traces ------------------------------------------------------------
  fig.add_trace(go_obj.Scatter( y=history.history['val_loss'], name="val_loss"),secondary_y=False,)
  fig.add_trace(go_obj.Scatter( y=history.history['loss'], name="loss"),secondary_y=False,)
  fig.add_trace(go_obj.Scatter( y=history.history['val_accuracy'], name="val accuracy"),secondary_y=True,)
  fig.add_trace(go_obj.Scatter( y=history.history['accuracy'], name="accuracy"),secondary_y=True,)

  # Adding figure title---------------------------------------------------------
  fig.update_layout(title_text="Loss/Accuracy of Final Model")
  # Set x-axis title
  fig.update_xaxes(title_text="Epoch")

  # Set y-axes titles-----------------------------------------------------------
  fig.update_yaxes(title_text="<b>primary</b> Loss", secondary_y=False)
  fig.update_yaxes(title_text="<b>secondary</b> Accuracy", secondary_y=True)
  fig.show()
  #============================================================================

In [ ]:
################################################################################
def validate_test_images_and_display(model,number_test_images):
  #funtion for prediting the 33 supplied test image's file name and predicted class
  test_image_data = []
  test_image_filenames = []
  IMG_SHAPE  = (224, 224)
  #-----------------------------------------------------------------------------
  for img_name in os.listdir(test_path):
    img = load_img(os.path.join(test_path, img_name), target_size = IMG_SHAPE)
    test_image_data.append(img_to_array(img, dtype = 'uint8'))
    test_image_filenames.append(img_name)
      
  test_image_data = np.array(test_image_data)/255
  #print(f'\nTotal testing images: {len(test_image_data)}')
  #-----------------------------------------------------------------------------
  test_pred = np.argmax(model.predict(test_image_data), axis = 1)

  class_name_lookup = {name: index for index, name in training_data.class_indices.items()}
  #for k, v in class_name_lookup.items():
  #  print(f"{k:2} : {v}")
  
  test_pred_classes = [class_name_lookup[i] for i in test_pred]

  data_frame=pd.DataFrame({
    "Filename": test_image_filenames,
    "Predicted classes": test_pred_classes
    })
  display (data_frame) # Displaying the dataframe 
  #-----------------------------------------------------------------------------
  #Display test images and its real label and predecited label here
  plt.subplots(nrows = 3, ncols = 4, figsize = (20, 15))

  for i in range(number_test_images):
    plt.subplot(3, 4, i + 1)
    plt.axis(False)
    plt.grid(False)
    plt.imshow(test_image_data[i])
    plt.title(f"True: {test_image_filenames[i][:-4]}\nPrediction:{test_pred_classes[i]}")
  plt.show()
  #-----------------------------------------------------------------------------
################################################################################

In [ ]:
class_name_lookup = {name: index for index, name in training_data.class_indices.items()}

In [ ]:
################################################################################
#Fuction to dispay confusion matrix based on the predections.
def Display_Confusion_Matrix(val_true,val_pred,class_name_lookup):
  # here printing the confustion matrix
  _, ax = plt.subplots(figsize = (20, 16))
  ax.grid(False)

  ConfusionMatrixDisplay(confusion_matrix(val_true, val_pred, labels = list(class_name_lookup.keys())),
                        display_labels = list(class_name_lookup.values())
                        ).plot(ax = ax, xticks_rotation = 'vertical')

  plt.savefig('confusion_matrix.jpg', bbox_inches = 'tight')
  plt.show()
################################################################################

In [ ]:
def Dispay_Classifiation_Report(val_true,val_pred,class_name_lookup):
################################################################################
# classifiation report printing here
  display_labels = list(class_name_lookup.values())
  print(classification_report(val_true,val_pred,target_names=class_name_lookup.values())) 
################################################################################

In [ ]:
target=validation_data.class_indices.keys() # getting the classs names for printing the ROC Graph
y_test =  validation_data.classes

In [ ]:
################################################################################
# function  roc auc score for multi-class datasets, since this dataset has 38 class
# funtion calling 2 times
def Multiclass_Roc_Auc_Scores(y_test, y_pred,min_class,max_class, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    i=0
    for (idx, c_label) in enumerate(target):
          i=i+1
          fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
          if i>=min_class and i<=max_class: 
            c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
def model_evalution(ev_model,value_data,model_size):
    ################################################################################
    #Model evalution : # Getting  validation accuracy and model size time to train
    ################################################################################
    start = perf_counter() 
    loss,Model_Accuracy = ev_model.evaluate(value_data, verbose = 0 )
    end = perf_counter() 
    ModelSize = model_size
    # let convert to MB
    ModelSize = ModelSize / (1024 * 1024)
    #-------------------------------------------------------------------------------
    print('Model Summary:')
    print('Model size(MB)                : {}'.format(ModelSize))
    print('Time on Validation data (sec) : {}'.format(end - start))
    print('Accuracy on validation data   : {}'.format(Model_Accuracy))
    print('Loss on validation data       : {}'.format(loss))
    #print('Time on Training Model  (sec) : {}'.format(end_training - start_training)

In [ ]:
## *** InceptionV3 **###
#Import the InceptionV3 library as shown below and
#add preprocessing layer to the front of InceptionV3
from tensorflow.keras.applications.inception_v3 import InceptionV3
#-------------------------------------------------------------------------------
inception = InceptionV3(input_shape=(224,224,3), weights='imagenet', include_top=False)
print("*** Building model with InceptionV3 with imagenet weights***")
model_inception = Sequential([
    inception,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.2),
    Dense(38, activation='softmax')
])


In [ ]:
######################################################################################
#Setting the same model name here, so all the functions will work without out any chage
model = model_inception #setting Inception model here
######################################################################################

In [ ]:
################################################################################
#Model compiling
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
################################################################################

In [ ]:
import keras.callbacks as callbacks
import time
early_stopping_cb = callbacks.EarlyStopping(monitor="loss", patience=3)

In [ ]:
start_training = perf_counter()
history = model.fit(
    training_data,
    validation_data = validation_data,
    epochs = 10, batch_size = 50,
    callbacks=[early_stopping_cb]
)
end_training= perf_counter()

In [ ]:
Display_Model_Loss_Accuracy(history) # calling funciton to print model accuracy graph

In [ ]:
number_test_images=12 # printing 12 images from the supplied test images
validate_test_images_and_display(model,number_test_images) # Prediting the supplied 33 images which is unkonw to the model

In [ ]:
#For confusion matrix
class_name_lookup = {name: index for index, name in training_data.class_indices.items()}
val_true = validation_data.classes
val_pred = np.argmax(model.predict(validation_data), axis = 1)
val_pred_float=model.predict(validation_data)
class_name_lookup = {name: index for index, name in training_data.class_indices.items()}

In [ ]:
Display_Confusion_Matrix(val_true,val_pred,class_name_lookup) # confusion-matrix

In [ ]:
model.save('final_model.h5') # saving the Model here, so that re-use the model later 

In [ ]:
model_evalution(model,validation_data,os.path.getsize('final_model.h5'))

In [ ]:
Dispay_Classifiation_Report(val_true,val_pred,class_name_lookup)